In [1]:
import tensorflow as tf

In [2]:
x = tf.Variable(12, name="x")
y = tf.Variable(5, name="y")
f = x*x*y + y + 2

Instructions for updating:
Colocations handled automatically by placer.


In [3]:
session = tf.Session()
session.run(x.initializer)
session.run(y.initializer)
res = session.run(f)
print(res)

727


In [4]:
session.close()

the other way around...

In [5]:
with tf.Session() as session :
    x.initializer.run()
    y.initializer.run()
    res = f.eval()

In [6]:
print(res)

727


In [7]:
session.close()

In [8]:
init = tf.global_variables_initializer()

with tf.Session() as session :
    init.run()
    res = f.eval

In [9]:
session = tf.InteractiveSession()
init.run()
res = f.eval()
print(res)

727


In [10]:
session.close()

Lifecycle of a Node evaluation

In [11]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = y + 3

In [12]:
with tf.Session() as s:
    print(y.eval())
    print(z.eval())

10
13


Linear Regression with TensorFlow

In [13]:
import numpy as np
from sklearn.datasets import fetch_california_housing

In [14]:
housing = fetch_california_housing()
m, n = housing.data.shape
housing_bias = np.c_[np.ones((m, 1)), housing.data]

In [15]:
x = tf.constant(housing_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
x_t = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(x_t, x)), x_t), y)

In [16]:
with tf.Session() as sess:
    theta_val = theta.eval()

Manual Computation of gradients for Batch Gradient Descent

In [17]:
epochs = 1000
learning_rate = 0.01

x = tf.constant(housing_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mean_square_error = tf.reduce_mean(tf.square(error), name = "mean_sq_error")
gradients = 2/m * tf.matmul(tf.transpose(x), error)
train_out = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, ", mean_sq_error = ", mean_square_error.eval())
        sess.run(train_out)
    best_theta = theta.eval()

Epoch:  0 , mean_sq_error =  1536031.8
Epoch:  100 , mean_sq_error =  nan
Epoch:  200 , mean_sq_error =  nan
Epoch:  300 , mean_sq_error =  nan
Epoch:  400 , mean_sq_error =  nan
Epoch:  500 , mean_sq_error =  nan
Epoch:  600 , mean_sq_error =  nan
Epoch:  700 , mean_sq_error =  nan
Epoch:  800 , mean_sq_error =  nan
Epoch:  900 , mean_sq_error =  nan


Using AutoDiff to compute gradients for Batch Gradient Descent

In [18]:
epochs = 1000
learning_rate = 0.01

x = tf.constant(housing_bias, dtype = tf.float32, name = "x")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(x, theta, name = "predictions")
error = y_pred - y
mean_square_error = tf.reduce_mean(tf.square(error), name = "mean_sq_error")
gradients = tf.gradients(mean_square_error, [theta])[0]
train_out = tf.assign(theta, theta - learning_rate * gradients)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for epoch in range(epochs):
        if epoch % 100 == 0:
            print("Epoch: ", epoch, ", mean_sq_error = ", mean_square_error.eval())
        sess.run(train_out)
    best_theta = theta.eval()

Epoch:  0 , mean_sq_error =  1301.037
Epoch:  100 , mean_sq_error =  nan
Epoch:  200 , mean_sq_error =  nan
Epoch:  300 , mean_sq_error =  nan
Epoch:  400 , mean_sq_error =  nan
Epoch:  500 , mean_sq_error =  nan
Epoch:  600 , mean_sq_error =  nan
Epoch:  700 , mean_sq_error =  nan
Epoch:  800 , mean_sq_error =  nan
Epoch:  900 , mean_sq_error =  nan


Implementation of Mini-Batch Gradient Descent

In [24]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [25]:
#construction phase with placeholder nodes
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
num_batches = int(np.ceil(m / batch_size))

In [27]:
#execution phase
def fetch_batch(epoch, batch_index, batch_size):   # load the data from disk; fetch mini-batches
    np.random.seed(epoch * num_batches + batch_index)
    indices = np.random.randint(m, size = batch_size)
    X_batch = housing_bias[indices]
    y_batch = housing.target.reshape(-1, 1)[indices]
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(epochs):
        for batch_index in range(num_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(train_out, feed_dict={X: X_batch, y: y_batch})
    best_theta = theta.eval()

Saving the trained model and saved checkpoints within to disk

In [36]:
num_epochs = 1000                                                                      
learning_rate = 0.01                                                                  

X = tf.constant(housing_bias, dtype = tf.float32, name = "X")            
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "y")            
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed = 42), name = "theta")
y_pred = tf.matmul(X, theta, name="predictions")                                      
error = y_pred - y                                                                    
mse = tf.reduce_mean(tf.square(error), name="mse")                                   
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)            
training_op = optimizer.minimize(mse)                                           

init = tf.global_variables_initializer()
saver = tf.train.Saver()      #creation of Saver node

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(num_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())                                
            save_path = saver.save(sess, "/tmp/my_model.ckpt") #save method to save the model with path of the checkpoint file
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

Epoch 0 MSE = 249103.42
Epoch 100 MSE = nan
Epoch 200 MSE = nan
Epoch 300 MSE = nan
Epoch 400 MSE = nan
Epoch 500 MSE = nan
Epoch 600 MSE = nan
Epoch 700 MSE = nan
Epoch 800 MSE = nan
Epoch 900 MSE = nan


Restoring the saved model from disk

In [39]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")   #restore method of the Saver object
    best_theta_restored = theta.eval()

INFO:tensorflow:Restoring parameters from /tmp/my_model_final.ckpt
